In [1]:
from market_analysis.models import Candle, UserProfile, Symbol, SortedStocksList, StrategyTimestamp
# from market_analysis import views
from django.contrib.auth.models import User
from heystox_intraday.select_stocks_for_trading import *
import pandas as pd
# from heystox_trade.settings import *
from datetime import datetime, timedelta
from heystox_intraday.intraday_fetchdata import *
from market_analysis.tasks.stock_data_import_tasks import *
from market_analysis.tasks.day_trading_tasks import *
from django.core.cache import cache, caches
from datetime import time, timedelta, datetime
from heystox_intraday.trading_indicator import *
import time

In [2]:
sorted_stocks = SortedStocksList.objects.filter(created_at__date=datetime.now().date())
sorted_stock = SortedStocksList.objects.get(symbol__symbol="tatamotors", created_at__date=datetime.now().date())
print(sorted_stock.created_at)
print(sorted_stock.entry_type)

2020-02-03 09:15:02.557074
SELL


In [3]:
# MACD Crossover
today_date = datetime.today().date()
df = get_macd_data(sorted_stock.symbol)
df.loc[(df["signal"] != df["signal"].shift()) & (df["signal"] == "BUY"), "signal"] = "BUY_CROSSOVER"
df.loc[(df["signal"] != df["signal"].shift()) & (df["signal"] == "SELL"), "signal"] = "SELL_CROSSOVER"
df = df.loc[df["date"] > str(today_date)]
try:
    if sorted_stock.entry_type == "SELL":
        last_crossover = df[df.signal.str.endswith("SELL_CROSSOVER")].iloc[-1]
    elif sorted_stock.entry_type == "BUY":
        last_crossover = df[df.signal.str.endswith("BUY_CROSSOVER")].iloc[-1]
except:
    last_crossover = None
if last_crossover is not None:
    df_after_last_crossover = df.loc[df["date"] > last_crossover.date]
    try:
        if last_crossover.signal == "SELL_CROSSOVER":
            crossover_signal = df_after_last_crossover.loc[(df.macd_diff <= -0.070)].iloc[0]
        elif last_crossover.signal == "BUY_CROSSOVER":
            crossover_signal = df_after_last_crossover.loc[(df.macd_diff >= 0.070)].iloc[0]
    except:
        crossover_signal = None
    if crossover_signal is not None:
        print("raju")
#         print(crossover_signal.item)
df.loc[(df.macd_diff <= -0.070)] # Signal
# df[df.signal.str.endswith("SELL_CROSSOVER")] # Crossover

raju


,candle_type,open_price,high_price,low_price,close_price,volume,total_buy_quantity,total_sell_quantity,date,macd,macd_signal,macd_diff,percentage,signal
151,M5,162.90,166.20,162.25,166.00,2956339,None,None,2020-02-03 09:15:00,-0.457049,0.211210,-0.668259,0.003054,SELL
152,M5,165.95,166.70,165.10,165.45,2138450,None,None,2020-02-03 09:20:00,-1.145589,-0.060150,-1.085439,0.012435,SELL
153,M5,165.40,165.45,163.35,163.50,1721755,None,None,2020-02-03 09:25:00,-1.827542,-0.413628,-1.413914,0.025840,SELL
154,M5,163.50,163.55,162.35,162.95,1495098,None,None,2020-02-03 09:30:00,-2.384883,-0.807879,-1.577004,0.037610,SELL
155,M5,162.95,164.30,162.95,164.15,1130714,None,None,2020-02-03 09:35:00,-2.698643,-1.186032,-1.512611,0.040820,SELL
156,M5,164.15,165.30,164.10,164.90,1228816,None,None,2020-02-03 09:40:00,-2.853883,-1.519602,-1.334281,0.038079,SELL
157,M5,164.90,166.30,164.85,166.15,1221769,None,None,2020-02-03 09:45:00,-2.843272,-1.784336,-1.058936,0.030108,SELL
158,M5,166.20,166.25,164.05,164.20,1151302,None,None,2020-02-03 09:50:00,-2.958112,-2.019091,-0.939021,0.027777,SELL
159,M5,164.25,165.80,164.00,165.75,776164,None,None,2020-02-03 09:55:00,-2.890730,-2.193419,-0.697310,0.020157,SELL
160,M5,165.80,167.70,165.75,167.40,1965544,None,None,2020-02-03 10:00:00,-2.673371,-2.289409,-0.383961,0.010265,SELL


In [4]:
# Shochastic Crossover
today_date = datetime.today().date()
df = get_stochastic_data(sorted_stock.symbol)
df.loc[(df["signal"] != df["signal"].shift()) & (df["signal"] == "BUY"), "signal"] = "BUY_CROSSOVER"
df.loc[(df["signal"] != df["signal"].shift()) & (df["signal"] == "SELL"), "signal"] = "SELL_CROSSOVER"
df = df.loc[df["date"] > str(today_date)]
try:
    if sorted_stock.entry_type == "SELL":
        last_crossover = df[df.signal.str.endswith("SELL_CROSSOVER")].iloc[-6]
    elif sorted_stock.entry_type == "BUY":
        last_crossover = df[df.signal.str.endswith("BUY_CROSSOVER")].iloc[-1]
except:
    last_crossover = None
if last_crossover is not None:
    df_after_last_crossover = df.loc[df["date"] > last_crossover.date]
    try:
        if last_crossover.signal == "SELL_CROSSOVER":
            crossover_signal = df_after_last_crossover.loc[(df.stoch_diff <= -22.70)].iloc[0]
        elif last_crossover.signal == "BUY_CROSSOVER":
            crossover_signal = df_after_last_crossover.loc[(df.stoch_diff >= 0.070)].iloc[0]
    except:
        crossover_signal = None
    if crossover_signal is not None and last_crossover is not None:
        print("RAJU")
# df.loc[(df.stoch_diff <= -22.70)] # Signal
df.head(50)

,candle_type,open_price,high_price,low_price,close_price,volume,total_buy_quantity,total_sell_quantity,date,stoch,stoch_signal,stoch_diff,percentage,signal
151,M5,162.90,166.20,162.25,166.00,2956339,None,None,2020-02-03 09:15:00,22.727273,35.318385,-12.591113,-2.861616,SELL
152,M5,165.95,166.70,165.10,165.45,2138450,None,None,2020-02-03 09:20:00,19.393939,23.468013,-4.074074,-0.790123,SELL
153,M5,165.40,165.45,163.35,163.50,1721755,None,None,2020-02-03 09:25:00,7.575758,16.565657,-8.989899,-0.681053,SELL
154,M5,163.50,163.55,162.35,162.95,1495098,None,None,2020-02-03 09:30:00,4.242424,10.404040,-6.161616,-0.261402,SELL
155,M5,162.95,164.30,162.95,164.15,1130714,None,None,2020-02-03 09:35:00,11.515152,7.777778,3.737374,0.430364,BUY_CROSSOVER
156,M5,164.15,165.30,164.10,164.90,1228816,None,None,2020-02-03 09:40:00,16.060606,10.606061,5.454545,0.876033,BUY
157,M5,164.90,166.30,164.85,166.15,1221769,None,None,2020-02-03 09:45:00,23.636364,17.070707,6.565657,1.551882,BUY
158,M5,166.20,166.25,164.05,164.20,1151302,None,None,2020-02-03 09:50:00,11.818182,17.171717,-5.353535,-0.632691,SELL_CROSSOVER
159,M5,164.25,165.80,164.00,165.75,776164,None,None,2020-02-03 09:55:00,21.212121,18.888889,2.323232,0.492807,BUY_CROSSOVER
160,M5,165.80,167.70,165.75,167.40,1965544,None,None,2020-02-03 10:00:00,31.212121,21.414141,9.797980,3.058157,BUY


In [5]:
# raju = ["adanient", "bhel", "dlf", "gmrinfra", "gail", "infratel", "ioc", "idea", "nhpc", "pnb", "relinfra", "recltd", "spencers", "tatapower", "vakrangee", "yesbank", "ujjivansfb", "granules", "mothersumi", "adanipower", "dishtv", "sail", "nationalum", "jswsteel", "federalbnk", "jindalstel", "idfcfirstb", "rvnl", "equitas"]
# fifth = date(2020,1,25)
# for r in raju:
#     symbol = Symbol.objects.get(symbol=r)
#     SortedStocksList.objects.create(symbol=symbol, entry_type="BUY")
# # Candle.objects.filter(date__date=fifth, symbol__symbol__in=raju)
# for candle in Candle.objects.filter(date__date=fifth, symbol__symbol__in=raju):
#     candle.date = candle.date + timedelta(1)
#     candle.save()

In [16]:
import requests

# url = "https://www1.nseindia.com/live_market/dynaContent/live_analysis/pre_open/nifty.json"
url = "https://www1.nseindia.com/live_market/dynaContent/live_analysis/pre_open/preOpenOrderBook.jsp?param=IOCEQN&symbol=IOC"
payload = {}
headers= {"Host": "www1.nseindia.com", 
            "Cache-Control": "max-age=0",
            "Upgrade-Insecure-Requests": "1", "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36",
             "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3",
             }
# print(response.text.encode('utf8'))
# url_request.text.encode("utf-8")
response = requests.get(url, headers=headers)


'\n \n\n \n\n\n\n<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 3.0//EN">\n<html>\n<head>\n<title>NSE - Pre Open Market</title>\n<link href="/live_market/resources/css/live_market.css" rel="stylesheet" type="text/css" />\n<script type="text/javascript" src="/common/js/jquery-1.4.4.min.js"></script>\n<script type="text/javascript" src="/live_market/resources/js/date.js"></script>\n<script>bazadebezolkohpepadr="1724186443"</script><script type="text/javascript" src="https://www1.nseindia.com/akam/11/66c4fc1f" defer></script></head>\n\n\n<body bgcolor="#ffffff" leftmargin="0" topmargin="0" marginheight="0" marginwidth="0" >\n\n\n<table cellpadding="0" cellspacing="0" border="0" width="100%" >\n<tr>\n<td>&nbsp;</td>\n\n<td width="100%">\n<table border="0" width="100%">\n\n\n\n\n\n\n\n\t<tr><td align="center">\n\t<table border="0" width="100%">\n\t<tr><td class="textFontCBold">Pre-Open Book - IOC</td></tr>\n\t<tr><td></td></tr>\n\t</table>\n\t</td></tr>\n\n\n\t<tr><td>\n\t<table bgcolor="#969696"

In [ ]:
# {'symbol': 'VISESHINFO',
#   'series': 'BE',
#   'xDt': '-',
#   'caAct': '-',
#   'iep': '0.10',
#   'chn': '0.05',
#   'perChn': '100.00',
#   'pCls': '0.05',
#   'trdQnty': '1,01,701',
#   'iVal': '0.10',
#   'mktCap': '9.62',
#   'yHigh': '0.10',
#   'yLow': '0.05',
#   'sumVal': '7,930.97',
#   'sumQnty': '52,80,743',
#   'finQnty': '1,01,701',
#   'sumfinQnty': '56,50,345'}